In [3]:
import sys

sys.path.insert(0, '..')

from app.processing.audio.SIWhisperModel import SIWhisperModel
from app.SIWeaviateClient import SIWeaviateClient
from app.processing.YoutubeProcessor import YoutubeProcessor

whisper = SIWhisperModel('medium', 'whisper-medium')



In [4]:
with SIWeaviateClient() as client:
    YoutubeProcessor(client=client, whisper=whisper, youtube_url="https://www.youtube.com/watch?v=_3n54Nrfq9I")

AUDIO CODEC aac


whisper-medium: 100%|██████████| 353.013/353.013 [00:16<00:00, 21.31s/s]


time_taken 17.14990782737732
